In [1]:
import pandas as pd
import pickle 
#from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
#from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
#from finrl.meta.data_processor import DataProcessor
import sys
sys.path.append("/findrl_ray/finenv")
sys.path.append("./FinRL")
from finenv.env_stocktrading import StockTradingEnv
from finenv.preprocessors import data_split
# load the DataFrame from a pickle file
#df = pd.read_pickle('dataset/nasdaq74_proc_test.pkl')

train = pd.read_csv('./FinRL/examples/train_data.csv')
train = train.set_index(train.columns[0])
train.index.names = ['']

TRAIN_START_DATE = '2016-01-01'
TRAIN_END_DATE = '2018-01-01'
TRADE_START_DATE = '2018-01-01'
TRADE_END_DATE = '2023-03-26'
#df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [2]:
INDICATORS = ['macd','boll_ub','boll_lb','rsi_30','cci_30','dx_30','close_30_sma','close_60_sma']

In [3]:
#train = data_split(df, TRAIN_START_DATE,TRAIN_END_DATE)
#trade = data_split(df, TRADE_START_DATE,TRADE_END_DATE)
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 82, State Space: 821


In [4]:
train.head()
train.dtypes

date             object
tic              object
open            float64
high            float64
low             float64
close           float64
volume          float64
day             float64
macd            float64
boll_ub         float64
boll_lb         float64
rsi_30          float64
cci_30          float64
dx_30           float64
close_30_sma    float64
close_60_sma    float64
vix             float64
turbulence      float64
dtype: object

In [5]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension
env_kwargs = {
    "hmax": 200,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [6]:
import psutil
import ray
ray._private.utils.get_system_memory = lambda: psutil.virtual_memory().total
from ray.tune.registry import register_env
from gymnasium.wrappers import EnvCompatibility

In [7]:
def env_creator(env_config):
    # env_config is passed as {} and defaults are set here
    df = env_config.get('df', train)
    hmax = env_config.get('hmax', 200)
    initial_amount = env_config.get('initial_amount', 1000000)
    num_stock_shares = env_config.get('num_stock_shares', [0] * stock_dimension)
    buy_cost_pct = env_config.get('buy_cost_pct', buy_cost_list)
    sell_cost_pct = env_config.get('sell_cost_pct', sell_cost_list)
    state_space = env_config.get('state_space', 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension)
    stock_dim = env_config.get('stock_dim', stock_dimension)
    tech_indicator_list = env_config.get('tech_indicator_list', INDICATORS)
    action_space = env_config.get('action_space', stock_dimension)
    reward_scaling = env_config.get('reward_scaling', 1e-3)

    return EnvCompatibility(StockTradingEnv(
        df=df,
        hmax=hmax,
        initial_amount=initial_amount,
        num_stock_shares=num_stock_shares,
        buy_cost_pct=buy_cost_pct,
        sell_cost_pct=sell_cost_pct,
        state_space=state_space,
        stock_dim=stock_dim,
        tech_indicator_list=tech_indicator_list,
        action_space=action_space,
        reward_scaling=reward_scaling
    ))

In [8]:
use_ddppo = False

In [9]:
from ray.rllib.agents import ppo
ray.shutdown()
print(f"ray is being initialized")
# ray.init(_temp_dir="FinRL/RLLIB/results", num_cpus=1, num_gpus=0)
# ray.init()

ray is being initialized


In [10]:
#if use_ddppo:
#    config = ppo.DDPPOConfig()
#else:
#    config = ppo.PPOConfig()

# Here set envirnonment parameters if different from default
#config = config.environment(
#    env_config={'hmax':200, 'initial_amount':1000000}
#)

# Training
#config = config.training()

# Resources
#config = config.resources(
#    num_gpus=0,
#    num_cpus_per_worker=1,
#    num_gpus_per_worker=0,
#    num_trainer_workers=1,
#    num_gpus_per_trainer_worker = 1,
#    num_cpus_per_trainer_worker = 1,
#)
# Framework
#config = config.framework(
#    framework="torch",
#)
# Rollouts
#config = config.rollouts(
#    num_rollout_workers=10,
#    num_envs_per_worker=1,
#)

In [11]:
config = ppo.PPOConfig()  
config = config.training(gamma=0.9, lr=0.001, kl_coeff=0.3)  
config = config.resources(num_gpus=0)  
config = config.rollouts(num_rollout_workers=6) 
config = config.framework(framework="torch")
config["model"]["fcnet_hiddens"] = [256, 256, 128]

In [12]:
# registering the environment to ray
register_env("finrl", env_creator)
if use_ddppo:
    trainer = ppo.DDPPOTrainer(env='finrl', config=config)
else:
    #trainer = ppo.PPOTrainer(env='finrl', config=config)
    trainer = config.build(env="finrl") 

2023-04-01 17:41:29,300	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(RolloutWorker pid=68396) 2023-04-01 17:41:32,877	WARNING env.py:156 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
(RolloutWorker pid=68396) 2023-04-01 17:41:32,877	WARNING env.py:166 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=68396) /Users/gibson/Downloads/ENTER/lib/python3.9/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=68396)   logger.warn("Casting input x to numpy array.")
(RolloutWorker pid=68397) /Users/gibson/Downloads/ENTER/lib/python3.9/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x

In [13]:
from tqdm.notebook import tqdm
# Train away -------------------------------------------------------------
total_episodes=10
agent_name = 'ppo'
ep = 0
results = []
bar = tqdm(total=total_episodes, desc="Episodes")
while ep <= total_episodes:
    results.append(trainer.train())
    ep += 1
    bar.update(n=1)
    if ep % 10 == 0:
        #cwd_checkpoint = "results/checkpoints/" + str(agent_name) + '_' + str(ep)
        cwd_checkpoint = "model/" + str('fcnet')
        trainer.save(cwd_checkpoint)
        print(f"Checkpoint saved in directory {cwd_checkpoint}")
bar.close()

(RolloutWorker pid=68401) /Users/gibson/Downloads/ENTER/lib/python3.9/site-packages/gymnasium/spaces/box.py:227: UserWarning: WARN: Casting input x to numpy array.
(RolloutWorker pid=68401)   logger.warn("Casting input x to numpy array.")


Checkpoint saved in directory results/checkpoints/ppo_10



In [14]:
## Mean Rewards ### 
#print(results[-1]['episode_reward_mean'])
print(results[-1])

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 25.92122264267296, 'cur_kl_coeff': 11.533007812499998, 'cur_lr': 0.0010000000000000005, 'total_loss': 9.203936920371106, 'policy_loss': 0.0007782782247710612, 'vf_loss': 9.021072874787034, 'vf_explained_var': 0.0017701937306311823, 'kl': 0.01578822968407559, 'entropy': 133.1522976126722, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 9765.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 44000, 'num_env_steps_trained': 44000, 'num_agent_steps_sampled': 44000, 'num_agent_steps_trained': 44000}, 'sampler_results': {'episode_reward_max': 2859.817037065104, 'episode_reward_min': -512.8893275215927, 'episode_reward_mean': 1477.3060917502971, 'episode_len_mean': 2354.0, 'episode_media': {}, 'episodes_this_iter': 6, 'policy_reward_min': {}, 'policy_re

In [16]:
# save the trained agent
#cwd_checkpoint = "model/" + str(agent_name) + '_' + str(ep)
cwd_checkpoint = "model/"
trainer.save(cwd_checkpoint)

'model/checkpoint_000011'